In [23]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import re
import os
import pickle
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy import stats
from statsmodels.stats.proportion import proportion_confint
%matplotlib inline
from matplotlib import pyplot as plt

In [18]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

In [19]:
def to_csr_matrix(X_matrix):
    
    # Конвертируем матрицу частот в разреженую матрицу по принципу BagOfWord
    # формируем координаты для данных
    #data = np.ones(len(X_matrix.flatten()), dtype=np.int)
    
    data = (X_matrix.flatten() != 0).astype(int)[X_matrix.flatten() != 0]
    col = X_matrix.flatten()[X_matrix.flatten() != 0]
    row = np.array([[n] * len(X_matrix[0]) for n in range(len(X_matrix))]).flatten()[X_matrix.flatten() != 0]
    
    X_sparse = csr_matrix((data, (row, col-1)), shape=(len(X_matrix), X_matrix.max()))
    return X_sparse
    

In [24]:
def prepare_sparse_train_set_window(path_to_csv_files, site_freq_path, 
                                    session_length=10, window_size=10):
    ''' ВАШ КОД ЗДЕСЬ'''
    files = glob(os.path.join(path_to_csv_files, '*.csv'))
    
    num_filter = r'[0-9]{4}'
    
    data = pd.DataFrame() #Таблица для данных по сессиям
    
    with open(site_freq_path, 'rb') as handle:
        site_freq = pickle.load(handle)
    
    for filename in files:
        #Собираем данные по сессиям 
        df = pd.read_csv(filename, parse_dates=True)
        df['user_id'] = int(re.findall(num_filter, filename)[-1])
        
        #Заменим названия сайтов на их ID
        df['site'] = df['site'].map(site_freq).apply(lambda x: x[0])
        
        data = data.append(df, ignore_index=True)
    return data
    #return to_csr_matrix(data.iloc[:, :-1].values), data.iloc[:, -1].values


In [26]:
data_test = prepare_sparse_train_set_window(os.path.join(PATH_TO_DATA,'3users'), 
                                                       os.path.join(PATH_TO_DATA,'site_freq_3users.pkl'),
                                                       session_length=5, window_size=3)

In [32]:
data_test({'timestamp':'2013-11-15 12:52:48', 'site':0, 'user_id':3})

TypeError: Can only append a Series if ignore_index=True or if the Series has a name

In [40]:
data_test

,timestamp,site,user_id
0,2013-11-15 09:28:17,10,1
1,2013-11-15 09:33:04,8,1
2,2013-11-15 09:52:48,8,1
3,2013-11-15 11:37:26,4,1
4,2013-11-15 11:40:32,8,1
5,2013-11-15 11:40:34,5,1
6,2013-11-15 11:40:35,1,1
7,2013-11-15 11:40:37,6,1
8,2013-11-15 11:40:40,2,1
9,2013-11-15 11:41:35,9,1


In [70]:
session_length=5
window_size=3
data_test = data_test[data_test.user_id == 1]
data_try = []

for i in range(0, len(data_test), window_size):
    data_try.append(data_test.iloc[i:i + session_length, 1].values.tolist())


In [71]:
data_try

[[10, 8, 8, 4, 8], [4, 8, 5, 1, 6], [1, 6, 2, 9, 10], [9, 10, 5, 5, 5], [5, 5]]